In [14]:
from os import listdir;
import numpy as np;
from keras.utils import np_utils;
import sys;

seqLength = 100;
path = "Stories/";
files = [f for f in listdir(path) if (".DS_Store" not in f) and (".ipynb_checkpoints" not in f)];
rawData = "";
startingData = [];
uniqueWords = [];
for f in files:
    text = open(path + f).read();
    
    text = text.replace("\t", " ");
#     text = text.replace("\n", " \n");
#     text = text.replace('!', " ! ");
#     text = text.replace('"', ' " ');
#     text = text.replace('#', " # "); 
#     text = text.replace("'", " ' "); 
#     text = text.replace('(', " ) "); 
#     text = text.replace(')', " ) "); 
#     text = text.replace(',', " , ");
#     text = text.replace('-', " - "); 
#     text = text.replace('.', " . "); 
#     text = text.replace('/', " / ");
#     text = text.replace(':', " : ");
#     text = text.replace(';', " ; "); 
#     text = text.replace('?', " ? ");
    text = text.replace('–', " - ");
    text = text.replace('—', " - ");
    text = text.replace('‘', " ' "); 
    text = text.replace('…', " . ");
    text = text.replace('ç', "c");
    text = text.replace('é', "e");
    text = text.lower();
    
    if "dan bigfootzerian parties in my butthole with his billionaire lifestyle" in text:
        print(f);
    
    words = text.split();
    for w in words:
        if w not in uniqueWords:
            uniqueWords.append(w)
    
    rawData += text;
    startingData.append(text[0:seqLength]);
    
chars = sorted(list(set(rawData)));
numChars = len(chars);
numData = len(rawData);

print("%d stories has been loaded"%len(files));
print("Number of different characters: %d"%numChars);
print("Different characters: " + str(chars));
print("Number of characters in the text: %d"%numData);

charInt = dict((c, i) for i,c in enumerate(chars));
intChar = dict((i, c) for i,c in enumerate(chars));

print("Converting input data to sequences...");
dataX = [];
dataY = [];
for i in range(numData - seqLength):
    seqIn = rawData[i:i+seqLength];
    seqOut = rawData[i+seqLength];
    
    dataX.append([charInt[c] for c in seqIn]);
    dataY.append(charInt[seqOut]);
    
numPatterns = len(dataX);
print("All data has been converted...");
print("Number of patterns: %d"%numPatterns);
print("Reshaping the data");
X = np.reshape(dataX, (numPatterns, seqLength, 1));
X = X/float(numChars);
Y = np_utils.to_categorical(dataY);

134 stories has been loaded
Number of different characters: 50
Different characters: ['\n', ' ', '!', '"', '#', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Number of characters in the text: 3041774
Converting input data to sequences...
All data has been converted...
Number of patterns: 3041674
Reshaping the data


In [16]:
sents = rawData.split(".");
print((max(sents, key=len)))

1037


In [44]:
wordSequence = [];
differentWords = rawData.split();
for i in range(0, len(differentWords), 100):
    if i < len(differentWords) - 100:
        wordSequence.append(" ".join(differentWords[i:i+100]));
    else:
        wordSequence.append(" ".join(differentWords[i:len(differentWords)]));
print(len(wordSequence[-1]));
print(wordSequence[-1]);

180
fierceness that sends shivers down my spine . " travis , will you marry me ? " my own sentient investment strategy asks . " of course i will , " i tell him . " of course i will . "


In [8]:
print(len(uniqueWords));
print(len(rawData.split()));
print(X.shape)

22976
557976
(3044078, 100, 1)


In [46]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(wordSequence)
sequences = tokenizer.texts_to_sequences(wordSequence)

In [57]:
from keras.preprocessing.sequence import pad_sequences

print(len(sequences[0]));
print(sequences[0])
print(len(tokenizer.word_index.items()))

data = pad_sequences(sequences, maxlen=100)
print(len(data[0]))
print(data[-1])

90
[11, 43, 7, 5, 345, 14, 64, 8973, 3981, 12, 52, 146, 164, 19, 1696, 6, 2, 800, 146, 816, 12, 4, 39, 164, 2, 4266, 36, 1, 241, 7, 9378, 1752, 3, 25, 19, 213, 4432, 7, 9629, 39, 5899, 47, 4663, 1609, 5401, 6616, 6, 464, 933, 8345, 28, 581, 9, 3, 16, 40, 2, 52, 544, 69, 2, 52, 5099, 22, 50, 70, 2, 137, 24, 1, 262, 7, 5, 432, 11, 717, 1598, 474, 11, 8, 2738, 310, 670, 97, 2, 763, 4015, 28, 1, 191]
12068
100
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0 11069
     9  2058 11028    31     5   766  5914   165    13  4833    10     5
   135   588  3809  3204   156     7   245     2   165     2    67    32
     7

In [6]:
count = 0;
for i in range(len(rawData) - 1):
    if rawData[i] == rawData[i + 1] and rawData[i] == " ":
        count += 1;
        
print(str(count / float(len(rawData))));

0.00022508757056577487


In [ ]:
from keras.models import Sequential;
from keras.layers import Dense, LSTM, Dropout;
import sys
import numpy

numOfCharacters = 1000;

model = Sequential();
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256));
model.add(Dropout(0.2));
model.add(Dense(Y.shape[1], activation='softmax'))
model.load_weights("AliceWeights/alltingle-weights-improvement-21-1.3086.hdf5");
model.compile(loss="categorical_crossentropy", optimizer="adam");
startText = startingData[np.random.randint(0, len(startingData))];
pattern = [charInt[c] for c in startText];
result = startText + "";
print("Starting Text: \"" + startText + "\"");

i = 0;
while i < numOfCharacters:
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(numChars)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = intChar[index]
    seq_in = [intChar[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
    i+= 1;
    
print("The end\n");